<a href="https://colab.research.google.com/github/donggeunahn/class-2022-spring/blob/main/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr
#gradio를 사용하기 위해서는 install과 import해야 함
#제목만 적으면 되는 것도 있음

### [Image classification](https://huggingface.co/tasks/image-classification)

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

In [ ]:
!pip install transformers
#transformer도 다운로드 해야됨

In [ ]:
#이름만 적어서 안되면 How to use 복붙
#image classification-->class가 1000개중에 확률이 가장 높은 부분
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
#image upload-->고양이 그림
image = Image.open(requests.get(url, stream=True).raw)
#image가 그림이라는 변수
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#pretrained는 행렬
#feature extraction을 할 수 있는 모델
#extracted 정보를 가지고 classification
inputs = feature_extractor(images=image, return_tensors="pt")
#feature를 뽑는 것-->object를 정의할 수 잇는 것을 뽑아냄
outputs = model(**inputs)
#image보다 input이 더 작음
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes-->1000개의 class 중 하나
predicted_class_idx = logits.argmax(-1).item()
#1000개 중에서 확률이 제일 큰 것을 찾아라
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

In [ ]:
#function을 만듦
def func (image):#image가 input으로 받아서 predicted class를 변수로
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
  #모델이 들어옴

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  #predicted class를 변수로 받음
  return predicted_class

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
#이 이름으로 다운 받아라
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

#Example을 위해 파일을 가져옴

0

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#function 넣고 input과 output 유형 정의
#input은 이미지, output은 text 유형으로, example도 추가(없어도 됨)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://10872.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fc2b5213e10>,
 'http://127.0.0.1:7861/',
 'https://10872.gradio.app')

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")
#mask 부분을 채워줌


Demo in Gradio

In [ ]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df
  #text를 입력 model이름은 unmasker
  #dictionary가 list로 되어 있음
  #dataframe화 됨

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()
#input은 text output은 dataframe 예문도 추가

### [Token classification](https://huggingface.co/tasks/token-classification)

natural language understanding-->natural language가 자유도가 너무 높다-->뽑아낼 수 있어야 함

POS tag 

NER(Named Entity Recognition)-->위치, 조직, 사람, 다양한 것 

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#두개의 pretrained를 불러옴
#tokenizer--> 전처리-->쪼개줌
#model-->각각의 단어들이 어떤 class인가-->NER
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"
#nlp가 모델-->pretrained를 모아줌
ner_results = nlp(example)
print(ner_results)
#두개의 dictionary가 list로 있음-->이럴때는 dataframe 바로 가능

Demo in Gradio

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df
  #dictionary가 list로 되어 있을때 가능

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)
-->주어진 문장이 있을때 얼마나 유사한가

model은 input을 바로 받아들일 수 없음-->중간단계 필요
raw data를 feature의 형태로 convert
vectors(embeddings)로 바꿔줘야 함-->similarity분석에 용이

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
#이 모델을 가지고 입력으로 위에 있는 sentences를 embedddings로 바꿔줌(feature의 형태)
print(embeddings)
#384개의 숫자 두개로 이루어진 list
#embedding for t1, embedding for t2
#벡터는 숫자들-->데이터-->similarity를 계산 가능

#각도로 유사성 파악 0도면 가장 유사 90도면 가장 다름
#cosine similarity cos(0)=1 cos(90)=0
#384차원이 되어도 두 벡터는 원점과 함께 삼각형을 만든다(
#-1에서 1까지의 값을 갖고 0에 가까울수록 유사도가 작다

In [ ]:
len(embeddings[1])
#384

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
#두 벡터가 이루는 cos 값-->첫번째 문장에 대한 384개의 숫자, 두번째 문장에 대한 384개의 숫자
cosine_scores

Demo in Gradio

In [ ]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  #list화
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [ ]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()